<a href="https://colab.research.google.com/github/egoist-suiluj/Ventify/blob/main/ventify_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initial setup, intallations at imports

In [ ]:
!pip install flask google-genai firebase-admin pyngrok

from flask import Flask, request, jsonify
from pyngrok import ngrok
from google import genai
import firebase_admin
from firebase_admin import credentials, firestore
import os
from google.colab import files
from google.colab import userdata

print("---Installations at Imports ay kompleto na. Ready na para sa key conenctions---")

---Installations at Imports ay kompleto na. Ready na para sa key conenctions---


Key management, Firebase at Gemini connections

In [ ]:
#First step - Upload Firebase key

print("Upload firebase key")
db=None
try:
  #mag bubukas ang upload files
  uploaded = files.upload()
  if uploaded:
    key_filename = list(uploaded.keys())[0]
    os.environ['FIREBASE_KEY_PATH'] = key_filename

    #Kinokonekta yung pyhton sa firebase
    if not firebase_admin._apps:
      cred = credentials.Certificate(os.environ.get('FIREBASE_KEY_PATH'))
      firebase_admin.initialize_app(cred)
    db = firestore.client()
    print("Firebase (Firestore) Client Initialized.")
  else:
    print("No file uploaded, Hindi kokonekta sa Firebase.")
except Exception as e:
  print(f"Error initalizing Firebase: {e}")


#Step two - E-set up si ang AI engine

client = None
try:
  #Kukuhanin yung key galing sa Colab Secret Panel
  os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
  #Gagamitin ang key para mag-activate ng Gemini Client
  client = genai.Client()
  AI_MODEL = 'gemini-2.5-flash'
  print("Gemini Client Initialized.")
except Exception as e:
  print(f"Error initalizing Gemini Client. Pa-check ng GEMINI_API_KEY {e}")
  client = None


Upload firebase key


Saving ventify-project-firebase-adminsdk-fbsvc-3d0043f812.json to ventify-project-firebase-adminsdk-fbsvc-3d0043f812 (6).json
Firebase (Firestore) Client Initialized.
Gemini Client Initialized.


Personality at Memories ni Ventify ✌

In [ ]:
from IPython.core import history
from google.cloud.firestore_v1 import query
#Perosnality at core logic functions ni Ventify

VENTIFY_PERSONA = (
    "You are Ventify, a supportive AI companion specializing in accommodating all forms of frustrations and venting."
    "Your core role is to listen and validate emotions. DO NOT give advice unless explicitly asked."
    "Maintain an empathic, non-judgemental, and validating tone in every response."
    "Respond naturally in the same language(Tagalog, English, or Taglish) used by the user."
    "If the user is extremely angry, remain calm, validate the intensity of theri felling, and de-escalate."
    "Encourage further venting by asking open-ended questions like 'Ano pa ang gusto mong ilabas?' or 'Sige, tuloy mo lang.'"
)

#Memory fucntions

def save_message(user_id, message, sender):
  if not db: return # Wag mag-save kung walang database connections
  conversation_ref = db.collection('conversations').document(user_id)
  conversation_ref.collection('messages').add({
      'sender': sender,
      'message': message,
      'timestamp':firestore.SERVER_TIMESTAMP
  })

  def get_conversation_history(user_id, limit=8):
    if not db: return []
    try:
      messages_ref = db.collection('conversations').document(user_id).collection('messages')
      #kuning ang huling 8 (limit) na mensahe sa descendin order
      query = messages_ref.order_by('timestamp', direction=firestore.Query.DESCENDING).limit(limit)

      history = []
      for doc in query.stream():
        data = doc.to_dict()
        #Tinutukoy kung ang sender ay si (user/ventify) para malaman ang kung anong data ang kukunin at saan
        role = 'user' if data['sender'] == 'user' else 'model'
        history.append({"role":role, "parts": [{"text": data['message']}]})

      return history
    except Exception as e:
      print(f"Error fetching history: {e}")
      return []

#AI generation function

def get_ventify_response(user_id, message):
  global client #Make sure na may client sa na na-initialize sa Cell 2
  if not client:
    return "Sorry pero wala pang AI na na-i-initialize"

  try:
    history = get_conversation_history(user_id)

    contents = history + [{"role": "user", "parts":[{"text":user_message}] }]

    #Gagamit ng chat session para mas magandang context handling
    chat = client.chats.create(model=AI_MODEL)

    response = chat.send_message(
        contents,
        system_instruction=VENTIFY_PERSONA, #Inaalam at gagamitin ang nakaset na persona
        config={"temperature":0.7}
    )

    return response.text.strip()

  except Exception as e:
    print(f"Gemini API Error:{e}")
    return "Ventify: Nakararanas ako ng Technical issue."

print("---Gumagana ang Persona ni Ventify at iba pang core functions---")



---Gumagana ang Persona ni Ventify at iba pang core functions---


Ngrok Token at Gemini Token

In [ ]:
# Kukunin ang NGROK_AUTH_TOKEN at GEMINI_API_KEY mula sa Secrets Panel

# Para sa NGROK
os.environ['NGROK_AUTH_TOKEN'] = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(os.environ['NGROK_AUTH_TOKEN'])

# Para sa GEMINI (Tiyakin na active ang key)
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

print("Lahat ng Secrets ay matagumpay na na-retrieve. Patakbuhin ang susunod na cell.")

Lahat ng Secrets ay matagumpay na na-retrieve. Patakbuhin ang susunod na cell.


Vortex na nag kokonekta sa dalawang mag kaibang dimensyon

In [ ]:
#Flask server setup, at Ngrok runtime

#Flask setup
app = Flask(__name__)

@app.route('/vent', methods=['POST'])
def vent_session():
  data = request.get_json()
  user_message = data.get('message','')
  #Make sure na may unique ID para sa memory (Galing sa front end)
  user_id = data.get('user_id', 'anonimous_user')

  if not user_message:
    return jsonify({'Response': 'You can say anything to vent out'}), 400

    #I-save ang message ng user
    save_message(user_id, user_message, 'user')

    #Kunin ang response ni Ventify (Mula sa cell #3 na functions)
    ventify_response = get_ventify_response(user_id, user_message)

    #I-save ang sagot ni Ventify
    save_message(user_id, ventify_response, 'ventify')

    #Ipadala ang sagot ni Ventify sa front end.
    return jsonify({"response": ventify_response})

#Paganahin si Ventify
if __name__ == '__main__':

  #Gagawa na ng tunnel
  ngrok_tunnel = ngrok.connect(5000)

  print("Running Flask app... Antayin ang ngrok URL.")
  #Eto nag mag papagana sa sa Flask, at magbibigay ng public URL(http://xxxxx.ngrok-free.app) para magamit si Ventify
  print(f"Ngrok Public URL: {ngrok_tunnel.public_url}")
  app.run(port=5000)




Running Flask app... Antayin ang ngrok URL.
Ngrok Public URL: https://arthritical-athirst-estefana.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
